<a href="https://colab.research.google.com/github/ti-lei/data-course-sample/blob/main/%E3%80%8Csample_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Code

## 載入資料

In [3]:
# windows (!python -m)跟 在 colab(!wget) 有差
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz


Saved under All_Beauty.csv

Saved under meta_All_Beauty.json.gz


## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
# from yellowbrick.cluster import KElbowVisualizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
metadata = getDF('/content/meta_All_Beauty.json.gz')
# 原始資料沒有 header 另外命名給他
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)
# 輸出metada 到csv
# metadata.to_csv('metada.csv',index=False)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User01\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 資料整理

In [2]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')
# 把 unixReviewTime drop
ratings.drop(columns=['unixReviewTime'],inplace = True)

## 資料切分

In [3]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
# agg(list)會把所有除了reviewerID的欄位用陣列包住
# to_dict 會把 dataframe 轉成字典(一列一個字典) 並用 陣列包住
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_trainings_by_user = ratings_trainings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
# 把 [{'asin': ['B01DKQAXC0'], 'reviewerID': 'A100XQFWKQ30O2'},..] 變成 {A100XQFWKQ30O2:['B01DKQAXC0'],...}
# key: reviwerID value: asin
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
ratings_trainings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_trainings_by_user }

# users:[user1,user2....]
training_users = list(ratings_trainings_by_user.keys())
testing_users = list(ratings_testings_by_user.keys())

# 應該要測試 users 在測試的資料裡 是否有跟 訓練的資料重疊 如果沒有的話根本不知道其消費行為 根本隨便推薦:


# 文字處理

In [4]:
table = str.maketrans('', '', string.punctuation)
porter = PorterStemmer()

def remove_redudant(text):
    # 先把 list 解開合在一起變成string
    text = ''.join(text)
    # 把文字拆開來
    words = text.split()
    # 把文字對表格做相對應maping去除掉符號並包起來
    stripped = [w.translate(table) for w in words]
    # 去除非全部是英文的文字
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # 去除只有一字的狀況
    words = [word for word in words if len(word)!= 1]
    # 篩掉 stop words
    stripped = [w for w in words if w not in stopwords.words('english')]
    # 正規化字串變成小寫字體
    #正規化動詞
    stemmed = [porter.stem(w) for w in stripped]
#     print(stemmed)
    return ' '.join(stemmed)

# metadata['clean_description'] = 
metadata['clean_description'] = metadata['description'].apply(lambda x: remove_redudant(x))
metadata['clean_tilte'] = metadata['title'].apply(lambda x: remove_redudant(x))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
# 計算商品用標題所表示的 tfidf 矩陣
# df = metadata.drop_duplicates('title')
df = metadata.drop_duplicates('asin').reset_index()
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['clean_description'])

# 計算相似度矩陣

In [15]:

from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

mapping = pd.Series(df.index,index = df['clean_description'])
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# Rule Based

In [8]:
ratings_training_rulebased = ratings[
(ratings['DATE'] >= '2018-06-01') & 
(ratings['DATE'] <= '2018-09-01')
]
ratings_compare = ratings_training_rulebased.groupby(['asin'], as_index=False).size()

## method 5: nearly 3_months: reviewer_count
recommand_rulebased = ratings_compare.sort_values(by='size',ascending=False)['asin'][:15].to_list()
# rating_sorting

# 將list展開的function

In [9]:
import functools
import operator
# Transform irregular 2D list into a regular one.
def transform(nested_list):
    regular_list = []
    for ele in nested_list:
        if type(ele) is list:
            regular_list.append(ele)
        else:
            regular_list.append([ele])
    return regular_list


# 建立 content based的推薦

In [16]:
import math
import itertools
# 放入要推薦的產品 到字典裡
recommend = {}
#從測試集找到之前在訓練集使用者購買的商品
cosine_recommend_num = 10
for user in testing_users:
    if user in training_users:
        
        num_of_product = len(ratings_trainings_by_user[user])
        # 使用者之前只有購買一個商品
        if num_of_product==1:
            asin = ''.join(ratings_trainings_by_user[user])
#             print(asin)
            recommend.update({user:recommend_item(asin,k=10)})
            # 剩下的商品用 ruledbased 補上
            recommend[user].append(recommand_rulebased[:5])
            

        # 當使用者之前購買的商品超過一個
        else:
            # 超過一個的狀況 每個商品推薦相似的數量 要把 預計要推薦的商品 除以總購買商品的數量
            # 假設之前有買過3個商品 則 每個商品的推薦相似商品數就只能有3個 floor(cosine_recommend_num/3)
            num_of_recommend = math.floor(cosine_recommend_num/num_of_product)
            for asin in ratings_trainings_by_user[user]:
                # 先設定default 這樣在第一次回圈再跑的時候 在取值時就不會找不到key
                recommend.setdefault(user, [])
                recommend[user].append(recommend_item(asin,num_of_recommend))
                
            recommend[user].append(recommand_rulebased[:15-(num_of_recommend*num_of_product)])
        # 把字典裡的list 都拆開來
        # https://stackabuse.com/python-how-to-flatten-list-of-lists/
        recommend[user] = functools.reduce(operator.iconcat, transform(recommend[user]), [])
#     如果user不是在testing data 裡就直接使用之前rubased的推薦
#     參考之前同學的最好的推薦是使用最接近的一個月推薦
    else:
        recommend.update({user:recommand_rulebased})
# recommend_item('B00015XAQA')

## 產生推薦

In [18]:
def recommender(users):

    recommendations = {users[i]:recommand_rulebased for i in range(len(users))}
    return recommendations


ratings_by_user = recommender( testing_users)

## 結果評估

In [25]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 是測試資料裡 key:user value:商品ID 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

# 僅使用 rule_based 的推薦
print(evaluate(ratings_by_user, ratings_testings_by_user))
# 使用綜合的推薦
print(evaluate(recommend, ratings_testings_by_user))

0.15423728813559323
0.15593220338983052
